In [1]:
import cv2
import numpy as np
import csv
import math

In [31]:
map_image = cv2.imread('perumdos.png', cv2.IMREAD_COLOR)
map_image = cv2.resize(map_image, (1920, 1080))
img = map_image.copy()

coordinates = []

with open('coordinates.csv', mode='r') as file:
  reader = csv.reader(file)
  next(reader)
  for row in reader:
    x, y = int(row[1]), int(row[2])
    coordinates.append((x, y))
    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)

print(coordinates)

[(1201, 148), (1173, 156), (1133, 158), (1067, 167), (1099, 163), (1033, 174), (990, 187), (1207, 204), (1170, 220), (1107, 221), (1116, 199), (1085, 202), (1058, 221), (1042, 246), (1025, 226), (968, 213), (950, 215), (962, 194), (938, 229), (940, 258), (923, 261), (919, 234), (904, 261), (890, 271), (869, 276), (852, 283), (997, 241), (976, 247), (997, 259), (975, 263), (860, 320), (961, 287), (958, 314), (915, 357), (895, 331), (852, 367), (1220, 287), (1173, 292), (1228, 340), (1209, 351), (1178, 334), (1158, 351), (1124, 306), (1101, 309), (1046, 333), (1100, 368), (1037, 378), (1017, 342), (1239, 425), (1221, 447), (1213, 422), (1177, 434), (1113, 442), (1092, 460), (1047, 449), (1007, 442), (976, 447), (916, 424), (933, 447), (904, 444), (878, 463), (899, 486), (1249, 498), (1197, 510), (1135, 522), (1058, 536), (997, 541), (929, 557), (860, 564), (1257, 554), (1209, 555), (1131, 573), (1075, 578), (997, 586), (955, 597), (927, 603), (866, 608), (836, 608), (730, 553), (765, 504

In [40]:
def print_coordinates(event, x, y, flags, param):
	if event == cv2.EVENT_LBUTTONDOWN:
		coordinates.append((x, y))
		# print(f"[{x}, {y}],")
		cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
		cv2.imshow("Image", img)

cv2.imshow("Image", img)
cv2.setMouseCallback("Image", print_coordinates)

while True:
	key = cv2.waitKey(1) & 0xFF

	if key == ord('d'):
		if len(coordinates) > 0:
			coordinates.pop()
			img = map_image.copy()
			for coord in coordinates:
				cv2.circle(img, coord, 5, (0, 0, 255), -1)
			cv2.imshow("Image", img)

	elif key == 27:
		break

cv2.destroyAllWindows()

with open('coordinates.csv', mode='w', newline='') as file:
	writer = csv.writer(file)
	writer.writerow(['No', 'X', 'Y'])
	for i, coord in enumerate(coordinates):
		writer.writerow([i, coord[0], coord[1]])

print(coordinates)

[(1201, 148), (1173, 156), (1133, 158), (1067, 167), (1099, 163), (1033, 174), (990, 187), (1207, 204), (1170, 220), (1107, 221), (1116, 199), (1085, 202), (1058, 221), (1042, 246), (1025, 226), (968, 213), (950, 215), (962, 194), (938, 229), (940, 258), (923, 261), (919, 234), (904, 261), (890, 271), (869, 276), (852, 283), (997, 241), (976, 247), (997, 259), (975, 263), (860, 320), (961, 287), (958, 314), (915, 357), (895, 331), (852, 367), (1220, 287), (1173, 292), (1228, 340), (1209, 351), (1178, 334), (1158, 351), (1124, 306), (1101, 309), (1046, 333), (1100, 368), (1037, 378), (1017, 342), (1239, 425), (1221, 447), (1213, 422), (1177, 434), (1113, 442), (1092, 460), (1047, 449), (1007, 442), (976, 447), (916, 424), (933, 447), (904, 444), (878, 463), (899, 486), (1249, 498), (1197, 510), (1135, 522), (1058, 536), (997, 541), (929, 557), (860, 564), (1257, 554), (1209, 555), (1131, 573), (1075, 578), (997, 586), (955, 597), (927, 603), (866, 608), (836, 608), (730, 553), (765, 504

In [28]:
# Fungsi untuk menghitung jarak Euclidean antara dua titik (x1, y1) dan (x2, y2)
def euclidean_distance(p1, p2):
  return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

In [29]:
# Map Scale
x1, y1 = 658, 157
x2, y2 = 710, 148
scale = 28.8 / euclidean_distance((x1, y1), (x2, y2))

print(scale)

0.545732587858402


In [44]:
# Prim's Algorithm
from scipy.spatial import distance_matrix
import networkx as nx

distances = distance_matrix(coordinates, coordinates)

G = nx.Graph()
for i, coord1 in enumerate(coordinates):
  for j, coord2 in enumerate(coordinates):
    if i != j:
      dist = distances[i][j]
      G.add_edge(i, j, weight=dist)

mst = nx.minimum_spanning_tree(G)

print("Edges in MST:")
for edge in mst.edges:
  print(f"{edge[0]} -> {edge[1]} with distance {mst[edge[0]][edge[1]]['weight'] * scale:.2f}")
print(f"Total distance of MST: {sum(mst[edge[0]][edge[1]]['weight'] for edge in mst.edges) * scale:.2f}")

img_result_prim = cv2.imread('perumdos.png')
img_result_prim = cv2.resize(img_result_prim, (1920, 1080))

for index, node in enumerate(mst.nodes):
  cv2.circle(img_result_prim, coordinates[node], 5, (0, 0, 255), -1)
  cv2.putText(img_result_prim, str(index+1), coordinates[node], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst.edges:
  pt1 = coordinates[edge[0]]
  pt2 = coordinates[edge[1]]
  cv2.line(img_result_prim, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Prim Result", img_result_prim)
cv2.waitKey(0)
cv2.destroyAllWindows()

Edges in MST:
0 -> 1 with distance 15.89
0 -> 7 with distance 30.74
1 -> 2 with distance 21.86
2 -> 4 with distance 18.75
3 -> 4 with distance 17.60
3 -> 5 with distance 18.94
3 -> 11 with distance 21.48
6 -> 17 with distance 15.75
7 -> 8 with distance 22.00
9 -> 10 with distance 12.97
9 -> 11 with distance 15.86
11 -> 12 with distance 18.02
12 -> 13 with distance 16.20
13 -> 14 with distance 14.32
14 -> 26 with distance 17.34
15 -> 16 with distance 9.88
15 -> 17 with distance 10.87
15 -> 27 with distance 19.06
16 -> 18 with distance 10.06
18 -> 21 with distance 10.72
19 -> 20 with distance 9.42
20 -> 22 with distance 10.37
20 -> 21 with distance 14.90
22 -> 23 with distance 9.39
23 -> 24 with distance 11.78
24 -> 25 with distance 10.03
25 -> 30 with distance 20.66
25 -> 147 with distance 20.75
26 -> 28 with distance 9.82
26 -> 27 with distance 11.92
27 -> 29 with distance 8.75
29 -> 31 with distance 15.16
30 -> 34 with distance 20.02
30 -> 35 with distance 26.02
31 -> 32 with distance

In [45]:
# Kruskal's Algorithm
import cv2
import numpy as np
from scipy.spatial import distance_matrix
import networkx as nx

distances = distance_matrix(coordinates, coordinates)

edges = []
for i, coord1 in enumerate(coordinates):
  for j, coord2 in enumerate(coordinates):
    if i < j:
      dist = distances[i][j]
      edges.append((dist, i, j))

edges.sort()

parent = list(range(len(coordinates)))
rank = [0] * len(coordinates)

def find(x):
  if parent[x] != x:
    parent[x] = find(parent[x])
  return parent[x]

def union(x, y):
  rootX = find(x)
  rootY = find(y)

  if rootX != rootY:
    if rank[rootX] > rank[rootY]:
      parent[rootY] = rootX
    elif rank[rootX] < rank[rootY]:
      parent[rootX] = rootY
    else:
      parent[rootY] = rootX
      rank[rootX] += 1

mst_edges = []
total_weight = 0

for dist, u, v in edges:
  if find(u) != find(v):
    union(u, v)
    mst_edges.append((u, v, dist))
    total_weight += dist

print("Edges in MST:")
for edge in mst_edges:
  print(f"{edge[0]} -> {edge[1]} with distance {edge[2] * scale:.2f}")
print(f"Total distance of MST: {total_weight * scale:.2f}")

img_result_kruskal = cv2.imread('perumdos.png')
img_result_kruskal = cv2.resize(img_result_kruskal, (1920, 1080))

for index, node in enumerate(coordinates):
  cv2.circle(img_result_kruskal, node, 5, (0, 0, 255), -1)
  cv2.putText(img_result_kruskal, str(index+1), node, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst_edges:
  pt1 = coordinates[edge[0]]
  pt2 = coordinates[edge[1]]
  cv2.line(img_result_kruskal, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Kruskal Result", img_result_kruskal)
cv2.waitKey(0)
cv2.destroyAllWindows()

Edges in MST:
27 -> 29 with distance 8.75
22 -> 23 with distance 9.39
19 -> 20 with distance 9.42
26 -> 28 with distance 9.82
15 -> 16 with distance 9.88
24 -> 25 with distance 10.03
16 -> 18 with distance 10.06
20 -> 22 with distance 10.37
18 -> 21 with distance 10.72
15 -> 17 with distance 10.87
89 -> 91 with distance 11.67
23 -> 24 with distance 11.78
26 -> 27 with distance 11.92
38 -> 39 with distance 11.98
119 -> 120 with distance 12.01
97 -> 98 with distance 12.02
90 -> 92 with distance 12.12
136 -> 137 with distance 12.20
132 -> 133 with distance 12.26
42 -> 43 with distance 12.66
57 -> 59 with distance 12.73
121 -> 122 with distance 12.74
9 -> 10 with distance 12.97
89 -> 125 with distance 13.50
100 -> 101 with distance 13.74
95 -> 96 with distance 13.91
48 -> 50 with distance 14.28
13 -> 14 with distance 14.32
40 -> 41 with distance 14.32
49 -> 50 with distance 14.32
93 -> 94 with distance 14.74
31 -> 32 with distance 14.83
20 -> 21 with distance 14.90
104 -> 105 with distance

In [ ]:
# Displaying the result use np

image1 = img_result_prim.copy()
image2 = img_result_kruskal.copy()

# resize to 50% of the original size
image1 = cv2.resize(image1, )

compare_result = np.hstack((img_result_prim, img_result_kruskal))